In [ ]:
import re
! pip install spacy
import spacy
! pip install spacymoji
import spacymoji

! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_sm


import pandas as pd
! pip install sklearn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49732 sha256=7e7fee0919db05a4557343c4d2cf0a11bb056b748b69e9984f1e999f85d8625b
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [ ]:
# use spacy for tokenization ,'tagger','parser','ner'
nlp = spacy.load("en_core_web_sm",exclude=['ner','textcat'])
nlp.add_pipe('emoji', first=True)
# get default pattern for tokens that don't get split
re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)
# add your patterns (here: hashtags and in-word hyphens)
re_token_match = f"({re_token_match}|\w+-\w+)"
# overwrite token_match function of the tokenizer
nlp.tokenizer.token_match = re.compile(re_token_match).match
nlp.tokenizer.rules = {key: value for key, value in nlp.tokenizer.rules.items() if "'" not in key and "’" not in key and "‘" not in key and "'" not in key}



Text preprocessing function for machine learning models

In [ ]:
class TextPreprocessor:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer

  def __call__(self, text):
    tokens = []
    for token in self.tokenizer(text):
      if token._.is_emoji:
        tokens.append(token._.emoji_desc)
      elif token.like_url:
        pass
      elif token.like_email or token.like_num or (token.is_ascii is False) or token.is_space or token.is_currency or token.is_digit or token.pos_=="NUM":
        pass
      elif token.is_punct and token.text not in self.tokenizer.tokenizer.rules.keys():
        pass
      elif token.is_stop:
        pass
      elif token.text:
          tokens.append(token.lemma_)
    return tokens

In [ ]:
tp = TextPreprocessor(nlp)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My\ Drive/vaccine_tweets

/content/gdrive/My Drive/vaccine_tweets


# Prepare training data

In [ ]:
import pandas as pd
df = pd.read_csv('training.csv')

In [ ]:
corpus = df['text']

In [ ]:
# prepare data
from sklearn import preprocessing, utils

Y_strings = df['mandate_m']
le = preprocessing.LabelEncoder()
Y = le.fit_transform(Y_strings)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer = tp, ngram_range=(1,1))
X_vector = tfidf_vectorizer.fit_transform(corpus)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
encode_dic = dict(zip(le.classes_, range(len(list(le.classes_)))))

In [ ]:
encode_dic

{0: 0, 1: 1}

In [ ]:
df.shape[0]

3499

In [ ]:
import math
train_size = math.floor(df.shape[0]*0.7) # model trains its parameters
eval_size = math.floor(df.shape[0]*0.15) # model evaluates its performance by itself
# 0.15 test set
print(train_size)
print(eval_size)

2449
524


In [ ]:
# shuffle and make train/test splits
X_vector_train, X_vector_test = X_vector[:train_size+eval_size], X_vector[train_size+eval_size:]
Y_train, Y_test = Y[:train_size+eval_size], Y[train_size+eval_size:]

In [ ]:
Y_train

array([1, 0, 0, ..., 1, 0, 0])

# Training

## Logistic regression with elastic net

In [ ]:
from sklearn.linear_model import LogisticRegression
model_en = LogisticRegression(penalty='elasticnet',solver='saga',max_iter=1000,l1_ratio=0.5)
model_en.fit(X_vector_train, Y_train)

LogisticRegression(l1_ratio=0.5, max_iter=1000, penalty='elasticnet',
                   solver='saga')

In [ ]:
# check training set performance
train_pred_Y_en = model_en.predict(X_vector_train)

In [ ]:
print(sklearn.metrics.classification_report(Y_train,train_pred_Y_en))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      2230
           1       0.97      0.56      0.71       743

    accuracy                           0.89      2973
   macro avg       0.92      0.78      0.82      2973
weighted avg       0.90      0.89      0.88      2973



In [ ]:
# check test set performance
pred_test_Y_en = model_en.predict(X_vector_test)
print(sklearn.metrics.classification_report(Y_test,pred_test_Y_en))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92       385
           1       0.94      0.57      0.71       141

    accuracy                           0.87       526
   macro avg       0.90      0.78      0.81       526
weighted avg       0.88      0.87      0.86       526



In [ ]:
sklearn.metrics.confusion_matrix(Y_test,pred_test_Y_en)/526

array([[0.72243346, 0.0095057 ],
       [0.11596958, 0.15209125]])

## Support vector machine

In [ ]:
from sklearn import model_selection, naive_bayes, svm
model_SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model_SVM.fit(X_vector_train, Y_train)# predict the labels on validation dataset

SVC(gamma='auto', kernel='linear')

In [ ]:
# check test set performance
pred_test_Y_SVM = model_SVM.predict(X_vector_test)
print(sklearn.metrics.classification_report(Y_test,pred_test_Y_SVM))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       385
           1       0.92      0.72      0.80       141

    accuracy                           0.91       526
   macro avg       0.91      0.85      0.87       526
weighted avg       0.91      0.91      0.90       526



In [ ]:
sklearn.metrics.confusion_matrix(Y_test,pred_test_Y_SVM)/526

array([[0.7148289 , 0.01711027],
       [0.07604563, 0.19201521]])

## Naive bayes

In [ ]:
model_NB = naive_bayes.BernoulliNB()
model_NB.fit(X_vector_train, Y_train)# predict the labels on validation dataset

BernoulliNB()

In [ ]:
# check test set performance
pred_test_Y_NB = model_NB.predict(X_vector_test)
print(sklearn.metrics.classification_report(Y_test,pred_test_Y_NB))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       385
           1       0.87      0.49      0.63       141

    accuracy                           0.84       526
   macro avg       0.86      0.73      0.76       526
weighted avg       0.85      0.84      0.83       526



In [ ]:
sklearn.metrics.confusion_matrix(Y_test,pred_test_Y_NB)/526

array([[0.71292776, 0.01901141],
       [0.13688213, 0.13117871]])